In [4]:
#imports
import pandas as pd
import numpy as np

In [9]:
#data import
main_df = pd.read_csv ('../data/cleaned_energy_data.csv')
main_df.head()

,country,year,population,life_expectancy,consumption_per_capita,consumption_per_GDP,ppp_2015USD,production_total,production_coal,production_natural_gas,...,imports_coal,imports_electricity,exports_crude_oil,exports_natural_gas,exports_coal,exports_electricity,emissions_co2_emissions,emissions_coal_and_coke,emissions_consumed_natural_gas,emissions_petroleum_and_other_liquids
0,World,1992,5.481774e+06,65.769659,64.426755,6.923158,51013.270526,353.718416,92.558725,76.984223,...,481679.269000,400.464479,30667.877,17854.85234,494096.4675,400.795117,22529.424264,8587.186368,4076.853622,9865.384274
1,Afghanistan,1992,1.448550e+04,51.641000,2.159523,1.042253,30.013600,0.016143,0.000158,0.011092,...,0.000000,0.131000,0.000,0.00000,0.0000,0.000000,1.704078,0.015551,0.588528,1.100000
2,Albania,1992,3.245890e+03,71.802000,25.937798,8.087302,10.410300,0.065519,0.003415,0.003760,...,104.719450,0.104000,0.000,0.00000,0.0000,0.560000,3.901890,0.502372,0.199519,3.200000
3,Algeria,1992,2.702830e+04,67.575000,48.164981,5.592648,232.773000,5.031036,0.000380,2.325281,...,1139.243484,0.125000,279.400,1240.61590,0.0000,1.061000,74.390488,2.687475,45.703013,26.000000
4,Angola,1992,1.265740e+04,45.230000,6.689229,1.423070,59.496900,1.152349,0.000000,0.021076,...,0.000000,0.000000,508.400,0.00000,0.0000,0.000000,5.218203,0.000000,1.118203,4.100000


In [11]:
#checking dimensions
main_df.shape

(4724, 33)

In [16]:
set(main_df['country'])

{'Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo-Brazzaville',
 'Congo-Kinshasa',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Côte d’Ivoire',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia, The',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hong Kong',


In [19]:
#function to filter oil exporting countries 
def oil_exp_countries (country_name):
    if country_name in ['Algeria', 'Angola', 'Congo-Brazzaville', 'Equatorial Guinea', 'Gabon', 'Iran', 
                        'Iraq', 'Kuwait', 'Libya', 'Nigeria', 'Saudi Arabia', 'United Arab Emirates',
                       'Venezuela', 'United States', 'Russia', 'China' ]:
        return 1
    else:
        return 0
    

In [22]:
#applying funtion to dataframe
main_df ['oil_exporting']= main_df.apply(lambda x:oil_exp_countries(x['country']), axis =1)

In [34]:
#crosschecking number of countries
oil_exporting= main_df[main_df['oil_exporting']== 1]
len(set(oil_exporting['country']))

16

16